# Tone Generation: Illusory Pitch

In [ ]:
import numpy as np
import librosa as lba
import soundfile as sf
from random import random

In [ ]:
stimuli_folder = '../stimuli/'

sr = 44100
tone_duration = 250

rise_duration = 10
rise_length = int(rise_duration * sr / 1000)

perc_duration = 240
perc_length = int(perc_duration * sr / 1000)

fade_duration = 10
fade_length = int(fade_duration * sr / 1000)

tones = {

    'A5+': 884,
    'A5': 880,
    'A5-': 876,
    
    'A4+': 446,
    'A4': 440, # Practice trial shifts are three times as large as others
    'A4-': 434,
    
    'A3+': 221,
    'A3': 220,
    'A3-': 219
}

# Generate Tones

In [ ]:
for pitch in tones:

    f = tones[pitch]
    
    # Generate fundamental frequency (co)sine wave
    tone = lba.tone(f, sr=sr, duration=tone_duration / 1000)
    
    # Add first three harmonics with slope of -6 db/half amplitude per octave
    for i in range(3):
        phase = random() * 2 * np.pi
        tone += lba.tone(f * (i + 2), sr=sr, duration=tone_duration / 1000, phi=phase) / (i + 2)
    
    # Rescale waveform to range [-1, 1] to prevent clipping
    tone /= np.abs(tone).max()

    # Apply exponential fade to create percussive envelope
    tone[-perc_length:] *= np.geomspace(1, .01, perc_length)
    # Apply short linear fade to ending so that amplitude fades to 0
    tone[-fade_length:] *= np.linspace(1, 0, fade_length)
    # Apply sharp linear rise to start of tone
    tone[:rise_length] *= np.linspace(0, 1, rise_length)
    
    # Save tone
    sf.write('../stimuli/tones/tone%s.wav' % pitch, tone, 44100)

## (Balance tones in Audacity now, then continue below)

# Generate Sequences

In [ ]:
base_intervals = 5
mod_intervals = 1
ntones = base_intervals + mod_intervals + 1

tempo = 500
offsets = [-.15, 0, .15]

for pitch in tones:

    if '+' in pitch or '-' in pitch:
        continue

    # Load normalized tone
    base_tone, _ = lba.load(stimuli_folder + 'tones/tone%s-normed.wav' % pitch, sr=sr)
    
    for shift in ('-', '+'):
        
        # Load probe tone
        tone, _ = lba.load(stimuli_folder + 'tones/tone%s%s-normed.wav' % (pitch, shift), sr=sr)
        
        for offset in offsets:
            
                # Create array of appropriate length to hold audio sequence
                ms_ioi = tempo * (1 + offset) / 1000  # Milisecond interval preceding probe tone
                sequence = np.zeros(int(np.ceil(ntones * max(tempo / 1000, ms_ioi) * sr)), dtype=np.float32)

                # Insert tones at appropriate locations
                for i in range(ntones):

                    # First several tones are spaced by the base IOI
                    if i <= base_intervals:
                        start = i * tempo / 1000 * sr
                        start = int(np.ceil(start))

                        # Place a copy of the tone in the proper location
                        base_tone_end = start + base_tone.shape[0]
                        sequence[start:base_tone_end] = base_tone

                    # Final tone(s) is/are preceded by the modified IOI  
                    else:
                        start = sr * (((base_intervals) * tempo / 1000) + ((i - base_intervals) * ms_ioi))
                        start = int(np.ceil(start))

                        # The tone sequence ends with a click; the click sequence ends with a tone
                        tone_end = start + tone.shape[0]
                        sequence[start:tone_end] = tone

                # Cut silence from the end of the sequence
                sequence = np.trim_zeros(sequence, 'b')

                # Save sequences to WAV file
                sf.write(stimuli_folder + 'sequence_%s%s_%i_%i.wav' % (pitch, shift, tempo, offset * 100), sequence, sr)